In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('ThisOne.csv')
df.head()

,problem_set,problem_set_name,User ID,condition,ExperiencedCondition,Saw Video,Class ID,Class Section ID,Class Grade,Teacher ID,...,Z-Scored Mastery Speed,Prior Homework Assigned,Prior Homework Count,Prior Homework Completion Count,Prior Homework Percent Completion,Prior Class Homework Percent Completion,Z-Scored HW Mastery Speed,complete,log(count),problem_count
0,226210,Multiplying Mixed Numbers 5.NF.B.4a EX,172777,E,False,NaN,31189,NaN,"""8""",42999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.602060,4
1,226210,Multiplying Mixed Numbers 5.NF.B.4a EX,175658,E,True,NaN,30642,54689.0,"""7""",68444,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,22
2,226210,Multiplying Mixed Numbers 5.NF.B.4a EX,175669,E,True,NaN,29270,51191.0,"""6""",59965,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.041393,11
3,226210,Multiplying Mixed Numbers 5.NF.B.4a EX,176151,E,True,NaN,29270,51191.0,"""6""",59965,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.000000,10
4,226210,Multiplying Mixed Numbers 5.NF.B.4a EX,176165,E,True,NaN,29270,51191.0,"""6""",59965,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,6


In [3]:
df['problem_set'].value_counts()

303899    1917
243393    1864
293151    1330
226210    1204
263115     754
263015     704
237447     695
241622     627
255116     545
246647     540
263052     523
246627     515
259379     497
246482     457
377658     429
263109     428
377938     426
256017     412
250476     402
241501     381
256027     335
263057     136
Name: problem_set, dtype: int64

In [4]:
df['ExperiencedCondition'].value_counts()

True     10761
False     4360
Name: ExperiencedCondition, dtype: int64

In [5]:
print len(df[df['condition']=='E'])
print len(df[(df['condition']=='E')&(df['ExperiencedCondition'] == True)])

7532
5336


In [6]:
print len(df[df['condition']=='C'])
print len(df[(df['condition']=='C')&(df['ExperiencedCondition'] == True)])

7589
5425


### generate training and testing data for each experiment.

In [11]:
cfr_df = pd.DataFrame()
is_video = True
problem_set_id = 259379
video_text_df = df[df['problem_set'] == problem_set_id]

print len(video_text_df)

if is_video:
    video_text_df = video_text_df[((video_text_df['Saw Video'] == 1) & \
                           (video_text_df['ExperiencedCondition'] == True)&(video_text_df['condition']=='E'))\
              | ((video_text_df['Saw Video'] == 1) &(video_text_df['ExperiencedCondition'] == True)&(video_text_df['condition']=='C'))]
    cfr_df['user_id'] = video_text_df['User ID']
    cfr_df['condition'] = ((video_text_df['Saw Video'] == 1) & \
                           (video_text_df['ExperiencedCondition'] == True)&(video_text_df['condition']=='E')).astype(int)
else:
    video_text_df = video_text_df[((video_text_df['ExperiencedCondition'] == True)&(video_text_df['condition']=='E'))\
              | ((video_text_df['ExperiencedCondition'] == True)&(video_text_df['condition']=='C'))]
    #print video_text_df.head()
    cfr_df['user_id'] = video_text_df['User ID']
    cfr_df['condition'] = np.where((video_text_df['condition']=='E')&\
                                   (video_text_df['ExperiencedCondition']==True), 1, 0)

cfr_df['yf'] = video_text_df['complete']

cfr_df['ycf'] = np.NaN

cols = ['Prior Problem Count', 'Prior Correct Count', 'Prior Assignments Assigned', \
        'Prior Assignment Count', 'Prior Homework Assigned', 'Prior Homework Count', \
        'Prior Completion Count', 'Prior Homework Completion Count', 'problem_count']
cols_norm = map(lambda x: x+' zscore', cols)
for idx, col in enumerate(cols):
    video_text_df[cols_norm[idx]] = (video_text_df[col] - video_text_df[col].min())/(video_text_df[col].max() - video_text_df[col].min())

features_list = ['Prior Percent Correct', 'Prior Percent Completion', 'Z-Scored Mastery Speed', 
                 'Prior Homework Percent Completion', 
                 'Z-Scored HW Mastery Speed', 'Prior Class Homework Percent Completion', 
                 'Prior Class Percent Completion'] + cols_norm

for ite in features_list:
    cfr_df[ite] = video_text_df[ite]

print cfr_df['condition'].value_counts()

msk = np.random.rand(len(cfr_df)) < 0.5

cfr_df.to_csv(str(problem_set_id) +'_exp.csv', header=False, na_rep='0')
print 'avg completion rate {}'.format(cfr_df['yf'].mean())
train_df = cfr_df[msk]
test_df = cfr_df[~msk]
print len(train_df)
print len(test_df)
train_df.to_csv(str(problem_set_id) +'_train_exp.csv', header=False, na_rep='0')
test_df.to_csv(str(problem_set_id) +'_test_exp.csv', header=False, na_rep='0')
print 'Train Data'
print 'Policy risk for control {}'.format(1-train_df[train_df['condition']==0]['yf'].mean())
print 'Policy risk for treatment {}'.format(1-train_df[train_df['condition']==1]['yf'].mean())
print 'Test Data'
print 'Policy risk for control {}'.format(1-test_df[test_df['condition']==0]['yf'].mean())
print 'Policy risk for treatment {}'.format(1-test_df[test_df['condition']==1]['yf'].mean())

497
1    156
0    145
Name: condition, dtype: int64
avg completion rate 0.445182724252
161
140
Train Data
Policy risk for control 0.578313253012
Policy risk for treatment 0.487179487179
Test Data
Policy risk for control 0.661290322581
Policy risk for treatment 0.512820512821


### Below is irrelevant. Keep it for future reference.

In [97]:
df[df['User ID'] == 96372]

,problem_set,problem_set_name,User ID,condition,ExperiencedCondition,Saw Video,Class ID,Class Section ID,Class Grade,Teacher ID,...,Z-Scored Mastery Speed,Prior Homework Assigned,Prior Homework Count,Prior Homework Completion Count,Prior Homework Percent Completion,Prior Class Homework Percent Completion,Z-Scored HW Mastery Speed,complete,log(count),problem_count
1223,237447,Understanding Vocabulary About Circles G-C.A.2 EX,96372,C,True,NaN,32278,58828.0,"""10""",162790,...,-0.449439,38.0,28.0,27.0,0.964286,0.961853,-0.481323,1,1.113943,13
6288,250476,Basic Logarithm Manipulation F-BF.B.5 EX,96372,E,False,1.0,32278,58828.0,"""10""",162790,...,-0.322099,50.0,38.0,37.0,0.973684,0.961905,-0.346814,1,0.477121,3
7233,256017,Intermediate Logarithm Manipulation F-BF.B.5 EX,96372,E,False,1.0,32278,58828.0,"""10""",162790,...,-0.359399,53.0,40.0,39.0,0.975000,0.958410,-0.387540,0,NaN,3
7645,256027,Solving ab^ct = d F-LE.A.4a EX,96372,E,False,1.0,32278,58828.0,"""10""",162790,...,-0.342592,52.0,45.0,44.0,0.977778,0.962929,-0.411135,1,0.477121,3
7985,259379,Finding Inverse Functions F-BF.B.4 EX,96372,C,False,1.0,32278,58828.0,"""10""",162790,...,-0.322099,52.0,38.0,37.0,0.973684,0.961754,-0.346814,1,0.477121,3
8478,263015,Composition of Functions F-BF.A.1c EX,96372,C,False,1.0,32278,58828.0,"""10""",162790,...,-0.311173,51.0,37.0,36.0,0.972973,0.962264,-0.335079,1,0.477121,3
9179,263052,Sequences F-BF.A.2 EX,96372,E,True,1.0,32278,58828.0,"""10""",162790,...,-0.422805,39.0,29.0,28.0,0.965517,0.962815,-0.449915,1,0.698970,5


In [103]:
96372 in df[df['ExperiencedCondition'] == True]['User ID'].unique().tolist()

True

In [104]:
len(df[df['ExperiencedCondition'] == True]['User ID'].unique().tolist())

6605

In [ ]:
s = ''
for ite in df['User ID'].unique().tolist():
    s += str(ite) + ','
s = s[:-1]
print s

In [49]:
exp_df = pd.read_csv('243393_exp.csv', header=None)

In [ ]:
s = ''
for ite in exp_df[1].tolist():
    s += str(ite) + ','
s = s[:-1]
print s 

In [ ]:
pred_df = pd.read_csv('../cfrnet/results/single_2-2_wass_VS0_20161121_233654-423/y_pred.csv', header=None, names=['factual', 'counterfactual'])

reset_cfr_df = cfr_df.reset_index()

pred_df['student_id'] = reset_cfr_df['user_id']
pred_df['condition'] = reset_cfr_df['condition']
pred_df['yf'] = reset_cfr_df['yf']

pred_df.head()

In [ ]:
pred_df['treatment_effect'] = np.where(pred_df['condition']==1, pred_df['factual']-pred_df['counterfactual'], \
                                       pred_df['counterfactual']-pred_df['factual'])

np.mean(pred_df['treatment_effect'])

In [ ]:
pred_df.head()